## 0. 라이브러리

In [28]:
import os
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification, TFBertModel
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model
import json
from soynlp.normalizer import *
from tqdm import tqdm
import re
import tensorflow as tf
import matplotlib.pyplot as plt
import random
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2
import numpy as np

## 1. 데이터 불러오기

In [68]:
train_data_path ="~/aiffel/dktc/data2/train5.csv"
train_data = pd.read_csv(train_data_path,index_col=0)
train_data

,class,conversation
idx,,
0,일반 대화,나 머리가 아침부터 아프네.\n타이레놀 먹어봐\n나는 타이레놀 만성이 되어서 안 들...
1,협박 대화,그 새끼 눈에 띄기만 하면 죽여버릴거니까 그런줄알아.\n아니 그래도 대화는 해봐야죠...
2,협박 대화,니가 그러고도 친구야?\n 그러는 너는?\n 헛소문 좀 퍼트리지마 \n 헛소문이라니...
3,갈취 대화,나 어제 너가 학원에서 선생님 지갑에서 돈 훔치는 거 봤어.\n어.? 아. \n처음...
4,일반 대화,**아 오늘까지 과제 제출이라며. 했어?\n아니 아직 안했어ㅠㅠ\n응? 어떡할려고 ...
...,...,...
6908,일반 대화,오늘 국민지원금으로 고기 파티했습니다.\n와 정말 맛있었겠어요\n정말 좋으셨겠어요\...
6909,기타 괴롭힘 대화,야. 너는 맨날 똑같은 옷만 입고 다니냐?\n아니야.\n더러워 진짜!\n냄새도 나냐...
6910,직장 내 괴롭힘 대화,씨 뭐해? \n아 저 지금 일 하는 중입니다\n바쁜가 보네 내가 접때 맡긴 건 다 ...


In [69]:
# 일반 대화 2000개 분리

normal_data = train_data[train_data["class"] == "일반 대화"]
not_normal_data = train_data[train_data["class"] != "일반 대화"]
print(len(normal_data))
print(len(not_normal_data))

2963
3950


In [70]:
train_data = pd.concat([not_normal_data, normal_data[:1000]], axis="rows")
normal_data = normal_data[1000:]
print(len(train_data))
print(len(normal_data))

4950
1963


## 2. 데이터 준비 (Data preparation)
### 2.1-1 전처리 함수 정의

In [71]:
def preprocess_sentence(sentence):
    # synolp
    emoticon_normalize(sentence)
    repeat_normalize(sentence)
    #sentence = re.sub(r'[^\w\s]', '', sentence)
    # base preprocess
    sentence = re.sub(r'([^a-zA-Zㄱ-ㅎ가-힣?.!,])', " ", sentence)
    sentence = re.sub(r'!+', '!', sentence)
    sentence = re.sub(r'\?+', '?', sentence)
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    # 엔터 구분 (\n)
    sentence = sentence.replace("\n", "<EOL>")
    sentence = sentence.strip()
    return sentence

### 2.1-2 전처리 함수 적용

In [72]:
# 학습할 문장이 담길 배열
sentences = []

for val in tqdm(train_data['conversation']):
    sentences.append(preprocess_sentence(val))
    
normal_sentences = []

for val in tqdm(normal_data['conversation']):
    normal_sentences.append(preprocess_sentence(val))

100%|██████████| 1963/1963 [00:00<00:00, 3334.97it/s]


### 2.2 최대 길이 지정

In [73]:
MAX_LEN = 200

### 2.3 class(label) 인코딩

In [74]:
from sklearn.preprocessing import LabelEncoder

CLASS_NAMES = ['협박 대화', '갈취 대화', '직장 내 괴롭힘 대화', '기타 괴롭힘 대화','일반 대화']

encoder = LabelEncoder()
encoder.fit(CLASS_NAMES)

train_data['class'] = encoder.transform(train_data['class'])
labels = train_data['class']

normal_data['class'] = encoder.transform(normal_data['class'])
normal_labels = normal_data["class"]

print(len(labels))
print(len(normal_labels))

4950
1963


In [75]:
class_mapping = {class_name: encoder.transform([class_name])[0] for class_name in CLASS_NAMES}
print("Class mapping:", class_mapping)

Class mapping: {'협박 대화': 4, '갈취 대화': 0, '직장 내 괴롭힘 대화': 3, '기타 괴롭힘 대화': 1, '일반 대화': 2}


In [76]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):
    
    input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []
    
    for example, label in tqdm(zip(examples, labels), total=len(examples)):
        # input_id는 워드 임베딩을 위한 문장의 정수 인코딩
        input_id = tokenizer.encode(example, 
                                    max_length=max_seq_len, 
                                    pad_to_max_length=True,
                                   )
        
        # attention_mask는 실제 단어가 위치하면 1, 패딩의 위치에는 0인 시퀀스
        padding_count = input_id.count(tokenizer.pad_token_id)
        attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count
        
        # token_type_id은 세그먼트 인코딩
        token_type_id = [0] * max_seq_len
        
        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)
        
        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        data_labels.append(label)
    
    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)
    
    data_labels = np.asarray(data_labels, dtype=np.int32)
    
    return (input_ids, attention_masks, token_type_ids), data_labels

### 2.4 train-val

In [77]:
train_sentences, val_sentences, train_labels, val_labels = train_test_split(
    sentences, labels, test_size=0.2, random_state=42, stratify=labels)

In [78]:
# train data 증강

def random_deletion(words, p=0.3):
    if len(words) == 1:
        return words

    new_words = []
    for word in words:
        r = random.uniform(0, 1)
        if r > p:
            new_words.append(word)

    if len(new_words) == 0:
        rand_int = random.randint(0, len(words) - 1)
        return [words[rand_int]]

    return "".join(new_words)

def swap_word(new_words):
    random_idx_1 = random.randint(0, len(new_words) - 1)
    random_idx_2 = random_idx_1
    counter = 0

    while random_idx_2 == random_idx_1:
        random_idx_2 = random.randint(0, len(new_words) - 1)
        counter += 1
        if counter > 3:
            return new_words

    new_words[random_idx_1], new_words[random_idx_2] = (
        new_words[random_idx_2],
        new_words[random_idx_1],
    )
    return new_words


def random_swap(words, n=3):
    new_words = words.copy()
    for _ in range(n):
        new_words = swap_word(new_words)

    return new_words


print("before data augmentation: ", len(train_sentences))

train_splted = pd.DataFrame({ "sentence": train_sentences, "class": train_labels })

train_splted = train_splted[train_splted["class"] != 2]

# random deletion
train_splted_rd = train_splted.copy()
train_splted_rd["sentence"] = train_splted_rd["sentence"].apply(random_deletion)

# random swap
train_splted_rs = train_splted.copy()

# with data augmentation
train_concated = pd.concat([train_splted , train_splted_rd , train_splted_rs])

print("after data augmentation: ", len(train_concated))

train_concated

before data augmentation:  3960
after data augmentation:  9480


,sentence,class
idx,,
3709,대리님 혹시 자료 보내주실 수 있을까요 ? 자료가 부족해서요 네 ? 나 알맞게 보냈...,3
5698,김회장 ! 오랜만이야 . 내가 놀라운 소식을 하나 들고 왔는데 말이야 . 지금 바쁘...,0
3094,또 뭘 처먹냐 뭐가 . 그만 좀 쳐먹어 씹돼지야 왜그래 . 너 걸어다닐때마다 바닥이...,1
255,지금 뭐하는건가 ? 꼼짝마 손들어 내가 그럴 것 같니 ? 딸에게 독약이라도 줄 거야...,4
3314,내가 술 좀 그만 먹으라고 했지 ? 죽을래 ? 술을 먹든 말든 신경 꺼 네가 술 먹...,4
...,...,...
3608,저 책임님 드릴 말씀이 있습니다 . 어 뭔데 ? 얘기해 제가 다음 주에 급하게 집에...,3
332,이거 누가 올린 보고서야 ! 오늘 까지 올리라고 하셔서 제가 아침에 올려놨는데요 ....,3
2541,왜 내가 부탁한 돈 아직도 안보냈어 ? 나 돈 없는거 알잖아 ? 그럼 우리 니가 보...,4


In [91]:
train_sentences, train_labels = train_concated["sentence"], train_concated["class"]
print(len(train_sentences))

9480


In [92]:
train_sentences = pd.concat([train_sentences, pd.Series(normal_sentences)], axis="rows")
train_labels = pd.concat([train_labels, normal_labels], axis="rows")

print(len(train_sentences))
print(len(train_labels))

11443
11443


In [98]:
print(train_sentences.head())

3709    대리님 혹시 자료 보내주실 수 있을까요 ? 자료가 부족해서요 네 ? 나 알맞게 보냈...
5698    김회장 ! 오랜만이야 . 내가 놀라운 소식을 하나 들고 왔는데 말이야 . 지금 바쁘...
3094    또 뭘 처먹냐 뭐가 . 그만 좀 쳐먹어 씹돼지야 왜그래 . 너 걸어다닐때마다 바닥이...
255     지금 뭐하는건가 ? 꼼짝마 손들어 내가 그럴 것 같니 ? 딸에게 독약이라도 줄 거야...
3314    내가 술 좀 그만 먹으라고 했지 ? 죽을래 ? 술을 먹든 말든 신경 꺼 네가 술 먹...
Name: sentence, dtype: object


In [100]:
# shuffle
train_sentences = train_sentences.reset_index(drop=True)
train_labels = train_labels.reset_index(drop=True)

train_sentences.name = "sentence"
train_labels.name = "class"

concat = pd.concat([train_sentences, train_labels], axis="columns")
concat = concat.sample(frac=1).reset_index(drop=True)

print(concat.head())
print(concat.tail())

                                            sentence  class
0  영지야 나 토익 학원 다니느라 돈을 다써서 그런데 만원만 빌려주라 응 ? 안돼 . ...      0
1  땡땡씨 얼굴 너무 크지않아 ? 맞아요 . 큰얼굴에 화장해놓고 자기가 이쁜줄 안다니까...      3
2  야 ! 너네 또 뛰었지 ! 저희 집이 뛴 거 아니라구요 너네잖아 ! 저희 지금 막 ...      4
3  야 저기 도움반 애 온다 쟤 진짜 짜증나 저번에 계속 나쫓아다님 지능이 낮아서 그런...      1
4  전 비오는날 빗소리 듣기가 너무 좋아요 이상하게 비오는 소리는 부침개 부치는 소리랑...      2
                                                sentence  class
11438  기 장님 급을언 주시나요알겠어새끼 기다 봐 지금 월급을못 받은 게 일이 넘었요 시는...      3
11439  오빠 나 돈 좀 빌려줘 친구가 돈 빌려달래 무슨 친구 얼마 ? 아니 그냥 좀 빌려줘...      0
11440  야 찐따야 . ? 지금 나 말하는거야 ? 그럼 너밖에 더 있겠냐 ? 왜 내가 찐따야...      1
11441  나 이번에 처음 해외여행 가면서 비행기도 처음 타봤어 어머 그래 ? 고등학교 때 보...      2
11442  인생 영화 추천좀 난 배우가 체질인듯 키키 멜로 배우 ? 액션 ? 내 인생 영화는 ...      2


## 3. 모델
### 3.1-1 토크나이저 정의

In [101]:
# BERT 토크나이저와 모델 준비
model_name = "klue/bert-base"

tokenizer = BertTokenizer.from_pretrained(model_name)
special_tokens_dict = {'additional_special_tokens': ['<EOL>']}
tokenizer.add_special_tokens(special_tokens_dict)

1

### 3.1-2 토크나이저 적용

In [103]:
# 데이터셋을 BERT 입력 형식으로 변환
X_train, y_train = convert_examples_to_features(
    concat["sentence"], concat["class"],
    max_seq_len=MAX_LEN, tokenizer=tokenizer
)
X_valid, y_valid = convert_examples_to_features(
    val_sentences, val_labels, 
    max_seq_len=MAX_LEN, tokenizer=tokenizer
)

# train_encodings = tokenizer(train_sentences, truncation=True, padding=True, max_length=MAX_LEN) # 뒤쪽에 패딩
# val_encodings = tokenizer(val_sentences, truncation=True, padding=True, max_length=MAX_LEN)

  0%|          | 0/11443 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2211: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 990/990 [00:00<00:00, 1110.03it/s]


### 3.2 모델 준비

In [104]:
class TFBertForMultiClassClassification(tf.keras.Model):
    def __init__(self, model_name, num_classes, dropout_rate=0.1):
        super(TFBertForMultiClassClassification, self).__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
        self.classifier = tf.keras.layers.Dense(num_classes,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                kernel_regularizer=l2(0.01),
                                                activation='softmax',
                                                name='classifier')

    def call(self, inputs):
        input_ids, attention_mask, token_type_ids = inputs
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids)
        cls_token = outputs[1]
        dropped = self.dropout(cls_token)
        prediction = self.classifier(dropped)
        return prediction

In [105]:
model = TFBertForMultiClassClassification(model_name, num_classes=5)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'bert.embeddings.position_ids', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

### 3.3 파라미터

In [106]:
BATCH_SIZE = 64
lr = 5e-5
EPOCH = 10

### 3.4 TF 데이터셋 생성

In [17]:
# TensorFlow 데이터셋 생성
# train_dataset = tf.data.Dataset.from_tensor_slices((
#     dict(X_train),
#     y_train
# )).shuffle(100).batch(BATCH_SIZE)

# val_dataset = tf.data.Dataset.from_tensor_slices((
#     dict(X_valid),
#     y_valid
# )).batch(BATCH_SIZE)


### 3.5 모델 컴파일

In [107]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

### 3.6 모델 훈련

### 3.6-1 콜백 설정

In [108]:
import datetime 

early_stopping = EarlyStopping(
    monitor='val_loss',    # 검증 손실을 모니터링
    patience=2,            # 3 에포크 동안 개선되지 않으면 중지
    restore_best_weights=True  # 최상의 가중치를 복원
)

now = datetime.datetime.now()
time = now.strftime("%y-%m-%d %H:%M")
data_type = 0

checkpoint = ModelCheckpoint(
    filepath=f'./models/klue_with_aug_weights_{data_type}_m250_{time}.keras',  # 모델 가중치를 저장할 파일 경로
    monitor='val_loss',        # 검증 손실을 모니터링
    save_best_only=True,       # 최상의 모델만 저장
    save_weights_only=True,    # 저장 (가중치)
    mode='min',                # 'val_loss'가 최소일 때 저장
    verbose=1                  # 저장 시 로그 출력
)

### 3.6-2 모델 훈련

In [109]:
model.fit(
    X_train, y_train, 
    validation_data=(X_valid, y_valid),
    epochs=EPOCH,
    callbacks=[early_stopping, checkpoint]
)

Epoch 1/10
358/358 [==============================] - 474s 1s/step - loss: 0.4328 - accuracy: 0.8778 - val_loss: 0.3638 - val_accuracy: 0.9111

Epoch 00001: val_loss improved from inf to 0.36384, saving model to ./models/klue_with_aug_weights_0_m250_24-06-26 15:53.keras
Epoch 2/10
358/358 [==============================] - 460s 1s/step - loss: 0.1657 - accuracy: 0.9744 - val_loss: 0.4500 - val_accuracy: 0.9020

Epoch 00002: val_loss did not improve from 0.36384
Epoch 3/10
358/358 [==============================] - 460s 1s/step - loss: 0.1234 - accuracy: 0.9869 - val_loss: 0.5060 - val_accuracy: 0.9000

Epoch 00003: val_loss did not improve from 0.36384


### 3.7 모델 평가

In [110]:
# 모델 평가
evaluation = model.evaluate(X_valid, y_valid)
print("평가 결과:", evaluation)

31/31 [==============================] - 13s 428ms/step - loss: 0.3638 - accuracy: 0.9111
평가 결과: [0.3638433516025543, 0.9111111164093018]


In [111]:
from sklearn.metrics import classification_report, f1_score, confusion_matrix
from sklearn.metrics import accuracy_score
import numpy as np 

def score(model, val):
    X, y = val
    # 실제 예측값 생성
    real_predictions = model.predict(X)

    # 예측값을 레이블로 변환
    real_predicted_labels = np.argmax(real_predictions, axis=1)

    # 정확도 계산
    real_accuracy = accuracy_score(y, real_predicted_labels)
    print(f"Real Accuracy: {real_accuracy:.4f}")

    # 분류 보고서 생성
    real_report = classification_report(y, real_predicted_labels, target_names=[f"Class {i}" for i in range(5)])
    print(real_report)

    # F1 스코어 계산
    real_f1 = f1_score(y, real_predicted_labels, average='weighted')
    print(f"\nWeighted F1 Score (based on real predictions): {real_f1:.4f}")

In [112]:
score(model, (X_valid, y_valid))

Real Accuracy: 0.9111
              precision    recall  f1-score   support

     Class 0       0.87      0.92      0.89       196
     Class 1       0.89      0.81      0.85       219
     Class 2       0.96      0.99      0.98       200
     Class 3       0.91      0.97      0.94       196
     Class 4       0.93      0.87      0.90       179

    accuracy                           0.91       990
   macro avg       0.91      0.91      0.91       990
weighted avg       0.91      0.91      0.91       990


Weighted F1 Score (based on real predictions): 0.9101


## 4. 모델 적용

In [113]:
import json

test_data_path = "/aiffel/aiffel/dktc/data/test.json"
test = pd.read_json(test_data_path).transpose()

In [114]:
import numpy as np

test_predict = []

for idx, value in test.iterrows():

    test_sentence = value["text"]
    
    test_encodings = tokenizer(test_sentence, truncation=True, padding="max_length", max_length=MAX_LEN, return_tensors="tf")

    test_predictions = model.predict(
        (test_encodings["input_ids"],
         test_encodings["attention_mask"],
         test_encodings["token_type_ids"])
    )
    test_class_probabilities = tf.nn.softmax(test_predictions, axis=-1).numpy() # [[0.13297564 0.8358507  0.00801584 0.02315779]]
    test_predicted_class = np.argmax(test_class_probabilities, axis=1) # [ 1 ]
    test_predict.append(test_predicted_class[0])

In [115]:
# {'협박 대화': 4, '갈취 대화': 0, '직장 내 괴롭힘 대화': 3, '기타 괴롭힘 대화': 1, '일반 대화': 2}
#   협박 대화 : 0,  갈취 대화 : 1,  직장 내 괴롭힘 대화 : 2,  기타 괴롭힘 대화 : 3,  일반 대화 : 4
def labelnum_to_text(x):
    if x == 0:
        return '01'
    if x == 1:
        return '03'
    if x == 2:
        return '04'
    if x == 3:
        return '02'
    if x == 4:
        return '00'

import datetime
    
submission = pd.read_csv("../data/new_submission.csv")
submission["class"] = [ labelnum_to_text(pred) for pred in test_predict ]

now = datetime.datetime.now()
filename = now.strftime("../sub/submission %y-%m-%d %H:%M.csv")

submission.to_csv(filename, index=False)
submit_file = pd.read_csv(filename)

print(submit_file.shape)
print(submit_file.head())

(500, 2)
  file_name  class
0     t_000      1
1     t_001      2
2     t_002      2
3     t_003      4
4     t_004      3


In [27]:
# submission 